# PISM TERRA for AGU 2025

In [ ]:
import s3fs
from pathlib import Path
import xarray as xr
import matplotlib.pylab as plt
import re
from functools import partial
from collections import OrderedDict, defaultdict
import numpy as np
import matplotlib as mpl
import warnings
import requests
from urllib.request import urlopen
from dateutil.parser import parse as date_parser
import pint_xarray
from dask.diagnostics import ProgressBar
import hyp3_sdk as sdk
import pandas as pd
import geopandas as gpd
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from matplotlib import cm, colors
import cmcrameri.cm as cmc

from pism_terra.processing import preprocess_config, normalize_cumulative_variables, standardize_variable_names

In [ ]:
def pick(files, pattern):
    for f in files:
        if pattern in Path(f).name:
            return f
    return None

In [ ]:
fs = s3fs.S3FileSystem(anon=True)  # or anon=True if public

PISM_CLOUD_BUCKET = 'hyp3-pism-cloud-test-contentbucket-zs9dctrqrlvx'

campaign = "era5_ec2"

user_id = "jhkennedy"
RGI_IDS = [
    'RGI2000-v7.0-C-01-03383',
    'RGI2000-v7.0-C-01-03102',
    'RGI2000-v7.0-C-01-01407',
    'RGI2000-v7.0-C-01-08332',
    'RGI2000-v7.0-C-01-14907',
    'RGI2000-v7.0-C-01-16106',
    'RGI2000-v7.0-C-01-14978',
    'RGI2000-v7.0-C-01-14612',
    'RGI2000-v7.0-C-01-04024',
    'RGI2000-v7.0-C-01-12784',
    'RGI2000-v7.0-C-01-04374',
    'RGI2000-v7.0-C-01-11818',
    'RGI2000-v7.0-C-01-08012',
    'RGI2000-v7.0-C-01-09429',
    'RGI2000-v7.0-C-01-16008',
    'RGI2000-v7.0-C-01-08153',
    'RGI2000-v7.0-C-01-03718',
    'RGI2000-v7.0-C-01-06260',
    'RGI2000-v7.0-C-01-05881',
    'RGI2000-v7.0-C-01-04928',
    'RGI2000-v7.0-C-01-05334',
    'RGI2000-v7.0-C-01-14209',
    'RGI2000-v7.0-C-01-13574',
    'RGI2000-v7.0-C-01-02832',
    'RGI2000-v7.0-C-01-07967',
    'RGI2000-v7.0-C-01-10901',
    'RGI2000-v7.0-C-01-00810',
    'RGI2000-v7.0-C-01-05069',
    'RGI2000-v7.0-C-01-04819',
    'RGI2000-v7.0-C-01-03017',
    'RGI2000-v7.0-C-01-07018']

JOB_NAMES = [rgi_id + "_" + campaign for rgi_id in RGI_IDS]

hyp3 = sdk.HyP3('https://pism-cloud-test.asf.alaska.edu')
jobs = sdk.Batch()
for job_name in JOB_NAMES:
    jobs += hyp3.find_jobs(name=job_name, user_id=user_id, job_type='PISM_TERRA_EXECUTE', start=date_parser('2025-12-11'))

s3_ids = {}
jobs_succeeded = 0
jobs_failed = 0
jobs_running = 0
jobs_pending = 0
for job in jobs:
    if job.job_type == 'PISM_TERRA_EXECUTE':
        rgi_id = job.name.split("_" + campaign)[0]
        job_id = job.job_id
        if job.status_code == "SUCCEEDED":
            s3_id = f's3://{PISM_CLOUD_BUCKET}/{job.job_id}'
            s3_ids[job_id] = {"rgi_id": rgi_id, "s3_id": s3_id}
            jobs_succeeded += 1
        elif job.status_code == "FAILED":
            jobs_failed += 1
        elif job.status_code == "RUNNING":
            jobs_running += 1
        elif job.status_code == "PENDING":
            jobs_pending += 1
        else:
            pass
        
print(f"PENDING: {jobs_pending}")
print(f"RUNNING: {jobs_running}")
print(f"SUCCEEDED: {jobs_succeeded}")
print(f"FAILED: {jobs_failed}")

In [ ]:
scalar_dict = {k: {} for k in s3_ids.keys()}
spatial_dict = {k: {} for k in s3_ids.keys()}

for job_id, s3_dict in s3_ids.items():
    s3_id = s3_dict["s3_id"]
    rgi_id = s3_dict["rgi_id"]
    prefix = f"{s3_id}/{rgi_id}/output/spatial/"
    files = fs.ls(prefix)  # returns 'bucket/key' style strings

    spatial_file = pick(files, "clipped_spatial_")
    scalar_file  = pick(files, "fldsum_spatial_")

    if spatial_file is not None:
        spatial_dict[job_id] = {"rgi_id": rgi_id, "url": f"s3://{spatial_file}"}
    if scalar_file is not None:
        scalar_dict[job_id] = {"rgi_id": rgi_id, "url": f"s3://{scalar_file}"}

scalar_files = [s["url"] for s in scalar_dict.values()]
spatial_files = [s["url"] for s in spatial_dict.values()]

In [ ]:
rgi_p = Path("/Users/andy/Google Drive/My Drive/Projects/terra/data/rgi/")

rgi = gpd.read_file("../data/rgi/rgi.gpkg")
rgi_ak = rgi[rgi["o1region"] == "01"]

processed_glaciers_ids = np.unique([v["rgi_id"] for v in scalar_dict.values()])
ak_total_area = rgi_ak["area_km2"].sum()
ak_processed_glaciers = rgi_ak[rgi_ak["rgi_id"].isin(processed_glaciers_ids)] 
ak_processed_area = ak_processed_glaciers["area_km2"].sum()
ak_processed_glaciers.to_file(rgi_p / "ak_processed_glaciers.gpkg")

print(f"{ak_processed_area / ak_total_area * 100}")

In [ ]:
mb_vars = {"ice_mass": "cumulative_mass_balance",
 "tendency_of_ice_mass_due_to_surface_mass_flux": "surface_mass_balance",
 "tendency_of_ice_mass_due_to_discharge": "ice_discharge",
 "tendency_of_ice_mass": "mass_balance"}

scalar_ds = xr.open_mfdataset(scalar_files, preprocess=preprocess_config, parallel=True, chunks="auto", engine="h5netcdf").sel({"time": slice("1980", "2025")})
scalar_ds = standardize_variable_names(scalar_ds, mb_vars)
scalar_ds = normalize_cumulative_variables(scalar_ds, "cumulative_mass_balance", reference_date="1980-01-01")
scalar_ds = scalar_ds[list(mb_vars.values())].pint.quantify()
scalar_ds["cumulative_mass_balance"] = scalar_ds["cumulative_mass_balance"].pint.to("Gt")

with ProgressBar():
    all_glaciers = scalar_ds.sum(dim="rgi_id").compute()

In [ ]:
percentiles = [0.05, 0.16, 0.50, 0.84, 0.95]
percentile_range_90 = (percentiles[-1] - percentiles[0]) * 100
percentile_range_68 = (percentiles[-2] - percentiles[1]) * 100
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", r"All-NaN (slice|axis) encountered")
    all_quantiles = {}
    for q in percentiles:
        all_quantiles[q] = all_glaciers.quantile(q, dim="exp_id", skipna=True)

In [ ]:
fontsize = 8
sim_alpha = 0.6
sim_cmap = ["#216778",  "#84cedc", "#ffffff", "#CC6677", "#882255"]
obs_alpha = 1.0
obs_cmap = ["0.8", "0.9"]
hist_cmap = ["#a6cee3", "#1f78b4"]

In [ ]:
add_median = True
p_var = "cumulative_mass_balance"
p = Path("/Users/andy/Google Drive/My Drive/Projects/terra/figures/")
figsize = (6.2, 6.2 * 9 / 16)
x_lim = [1980, 2025]
rcparams_white = {
    "axes.linewidth": 0.5,
    "axes.edgecolor": "white",
    "xtick.major.size": 2.0,
    "xtick.major.width": 0.25,
    "ytick.major.size": 2.0,
    "ytick.major.width": 0.25,
    "hatch.linewidth": 0.25,
    'font.weight': "bold",
    "font.size": fontsize,
    'axes.labelweight': "bold",
    'text.color': "white",
    'axes.labelcolor': "white",
    'xtick.color': "white",
    'ytick.color': "white",
}

with mpl.rc_context(rcparams_white):

    fig, ax = plt.subplots(1, 1, figsize=figsize)
    all_cis = []
    all_ci = ax.fill_between(
        all_quantiles[0.5].time,
        all_quantiles[percentiles[0]][p_var],
        all_quantiles[percentiles[-1]][p_var],
        alpha=sim_alpha,
        color=sim_cmap[0],
        lw=0,
        label=f"""{percentile_range_90:.0f}% credibility interval""",
    )
    all_cis.append(all_ci)
    all_ci = ax.fill_between(
        all_quantiles[0.5].time,
        all_quantiles[percentiles[1]][p_var],
        all_quantiles[percentiles[-2]][p_var],
        alpha=sim_alpha,
        color=sim_cmap[1],
        lw=0,
        label=f"""{percentile_range_68:.0f}% c.i.""",
    )
    all_cis.append(all_ci)
    
    if add_median:
        all_quantiles[percentiles[0]][p_var].plot(
            color=sim_cmap[2],
            add_legend=False,
            ax=ax,
            lw=0.5,
            ls="solid",
        )
        all_quantiles[percentiles[1]][p_var].plot(
            color=sim_cmap[2],
            add_legend=False,
            ax=ax,
            lw=0.5,
            ls="solid",
        )
        h  = all_quantiles[0.5][p_var].plot(
            color=sim_cmap[2],
            add_legend=False,
            ax=ax,
            lw=1,
            ls="solid",
            label="Median"
        )
        all_cis.append(h[0])
        all_quantiles[percentiles[-2]][p_var].plot(
            color=sim_cmap[2],
            add_legend=False,
            ax=ax,
            lw=0.5,
            ls="solid",
        )
        all_quantiles[percentiles[-1]][p_var].plot(
            color=sim_cmap[2],
            add_legend=False,
            ax=ax,
            lw=0.5,
            ls="solid",
        )
    l = ax.legend(handles=all_cis[::-1], loc="lower left")
    l.get_frame().set_linewidth(0.0)
    l.get_frame().set_alpha(0.0)
    ax.set_title("RGI Region 01 (Alaska)",  {"weight": "bold"})
    ax.set_ylabel("Cumulative mass change\nsince 1980 (Gt)")
    ax.set_xlim(
        np.datetime64(f"{x_lim[0]}-01-01"),
        np.datetime64(f"{x_lim[1]}-01-01"),
    )
    fig.tight_layout()
    fig.savefig(p / "all_glaciers.png", dpi=300, transparent=True)
    plt.close()
    del fig


In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", r"All-NaN (slice|axis) encountered")
    glacier_quantiles = []
    for q in [percentiles[0], 0.5, percentiles[1]]:
        glacier_quantiles.append(scalar_ds.quantile(q, dim="exp_id", skipna=True).expand_dims({"quantile": [q]}))
glacier_quantiles = xr.merge(glacier_quantiles)
with mpl.rc_context({"font.size": fontsize}):
    
    for rgi_id, glacier in glacier_quantiles.groupby("rgi_id"):
        s = glacier.squeeze()
        fig, ax = plt.subplots(1, 1, figsize=(6.4, 3.2))
        glacier_cis = []
        glacier_ci = ax.fill_between(
            s.sel({"quantile": 0.5}).time,
            s.sel({"quantile": percentiles[0]})[p_var],
            s.sel({"quantile": percentiles[-1]})[p_var],
            alpha=sim_alpha,
            color=sim_cmap[0],
            lw=0,
            label=f"""{percentile_range:.0f}% credibility interval""",
        )
        all_cis.append(all_ci)
        ax.set_title(rgi_id)
        ax.set_ylabel("Cumulative mass change\nsince 1980 (Gt)")
        ax.set_title("Alaska Glaciers")
        ax.set_xlim(
        np.datetime64(f"{x_lim[0]}-01-01"),
        np.datetime64(f"{x_lim[1]}-01-01"),
    )
        fig.savefig(p / f"{rgi_id}.png", dpi=300)
        plt.close()
        del fig

In [ ]:
add_median = False
with mpl.rc_context({"font.size": fontsize}):

    fig, axs = plt.subplots(3, 1, sharex=True, figsize=figsize)
    sim_cis = []

    for k, p_var in enumerate(["mass_balance", "surface_mass_balance", "ice_discharge"]):
        sim_ci = axs[k].fill_between(
            sim_quantiles[0.5].time,
            sim_quantiles[percentiles[0]][p_var],
            sim_quantiles[percentiles[1]][p_var],
            alpha=sim_alpha,
            color=sim_cmap[0],
            lw=0,
            label=f"""{percentile_range:.0f}% credibility interval""",
        )
        sim_cis.append(sim_ci)
    
    if add_median:
        sim_quantiles[percentiles[0]][p_var].plot(
            color=sim_cmap[1],
            add_legend=False,
            ax=ax,
            lw=0.25,
            ls="solid",
        )
        h  = sim_quantiles[0.5][p_var].plot(
            color=sim_cmap[1],
            add_legend=False,
            ax=ax,
            lw=1,
            ls="solid",
            label="Median"
        )
        sim_cis.append(h[0])
        sim_quantiles[percentiles[1]][p_var].plot(
            color=sim_cmap[1],
            add_legend=False,
            ax=ax,
            lw=0.25,
            ls="solid",
        )
    l = axs[-1].legend(handles=sim_cis, loc="lower left")
    l.get_frame().set_linewidth(0.0)
    l.get_frame().set_alpha(0.0)
    axs[-1].set_ylabel("Cumulative mass change\nsince 1978 (Gt)")
    axs[-1].set_xlim(
        np.datetime64(f"{x_lim[0]}-01-01"),
        np.datetime64(f"{x_lim[1]}-01-01"),
    )

    fig.savefig(p / "all_glaciers_fluxes.png", dpi=300, transparent=True)
    plt.close()
    del fig

In [ ]:
larsen_mean = -77
larsen_std = 11
all_glaciers.sel({"time": slice("1994", "2013")}).mean(dim="time").mass_balance

In [ ]:
by_rgi = defaultdict(list)

for job_id, info in spatial_dict.items():
    rgi_id = info["rgi_id"]
    url = info["url"]
    by_rgi[rgi_id].append(url)

In [ ]:
from dask.distributed import Client, progress
import xarray as xr

client = Client()  # or connect to your scheduler
print(f"Open client in browser: {client.dashboard_link}")

def compute_speed_for_rgi(rgi_id: str, urls: list[str]):
    ds = xr.open_mfdataset(
        urls,
        preprocess=preprocess_config,
        parallel=True,
        chunks="auto",
        engine="h5netcdf",
    )
    ds = ds.sel(time=slice("1980", "2025"))
    ds_last = ds.isel(time=-1)
    speed = ds_last["velsurf_mag"].median(dim="exp_id").squeeze()
    return speed

# submit futures
futures = {
    rgi_id: client.submit(compute_speed_for_rgi, rgi_id, urls)
    for rgi_id, urls in by_rgi.items()
}

# show progress bar for all futures
progress(list(futures.values()))

# gather results once done
results = {rgi_id: fut.result() for rgi_id, fut in futures.items()}

In [ ]:
all_rgi_files = Path("/Users/andy/Google Drive/My Drive/data/RGI7/RGI2000-v7/RGI2000-v7-G").rglob("*.shp")
all_rgi = pd.concat([gpd.read_file(f) for f in list(all_rgi_files)])

In [ ]:
def plot_hist(df, threshold, name):
    df = df[df["area_km2"] > threshold]
    data = df["area_km2"]
    xmin = max(1, np.nanmin(data[data > 0]))
    xmax = np.nanmax(data)
    lo, hi = np.floor(np.log10(xmin)), np.ceil(np.log10(xmax))
    bins = np.logspace(lo, hi, 11)  # 10 edges → 9 bars
    
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    
    fig, ax = plt.subplots(1, 1, figsize=(6.4, 3.0))
    counts, edges, patches = ax.hist(
        data,
        bins=bins,
        histtype="bar",
        rwidth=1.0,
        edgecolor="black",
        linewidth=0.25,
    )
    
    # ---- color bars: viridis, linear in area (km^2) ----
    cmap = mpl.colormaps.get_cmap("inferno")
    cmap = cmc.managua_r
    norm = colors.LogNorm(vmin=edges[0], vmax=edges[-1])  # linear mapping
    
    # Representative area per bar (geometric mean matches log-x placement)
    centers = np.sqrt(edges[:-1] * edges[1:])
    # If you prefer linear midpoints instead: centers = 0.5*(edges[:-1] + edges[1:])
    
    for p, a in zip(patches, centers):
        p.set_facecolor(cmap(norm(a)))
    
    # Optional colorbar
    sm = cm.ScalarMappable(norm=norm, cmap=cmap); sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
    cbar.set_label("Area (km²)")
    cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{int(x):,}"))

    area = sum(data)
    ax.set_title(f"{name}", {"weight": "bold"})
    # Axes styling
    ax.set_xscale("log"); ax.set_yscale("log")
    ax.set_xticks([1, 10, 100, 1000, 10000, 100000])
    ax.xaxis.set_major_formatter(formatter)
    ax.yaxis.set_major_formatter(formatter)
    ax.set_xlim([1, 100_000])
    ax.set_xlabel("Area (km$^{2}$)")
    ax.set_ylabel("Count (1)")
    for side in ("top", "right"):
        ax.spines[side].set_visible(False)
    ax.tick_params(top=False, right=False)
    
    # # (Optional) annotate counts:
    # for c, rect in zip(counts, patches):
    #     if c <= 0: continue
    #     x = rect.get_x() + rect.get_width()/2
    #     ax.annotate(f"{int(c):,}", (x, c),
    #                 xytext=(0, 4), textcoords="offset points",
    #                 ha="center", va="bottom", clip_on=False)
    
    fig.tight_layout()
    return fig
    

threshold = 0.001
fontsize = 12
rcparams_white = {
    "axes.linewidth": 0.75,
    "axes.edgecolor": "white",
    "xtick.major.size": 2.0,
    "xtick.major.width": 0.5,
    "ytick.major.size": 2.0,
    "ytick.major.width": 0.5,
    "hatch.linewidth": 0.5,
    'font.weight': "bold",
    "font.size": fontsize,
    'axes.labelweight': "bold",
    'text.color': "white",
    'axes.labelcolor': "white",
    'xtick.color': "white",
    'ytick.color': "white",
}
p = Path("/Users/andy/Google Drive/My Drive/Projects/terra/figures/")


with mpl.rc_context(rcparams_white):
    fig = plot_hist(all_rgi, threshold, "Size")
    fig.savefig(p / "rgi_size_distribution.png", dpi=300, transparent=True)

In [ ]:
with mpl.rc_context(rcparams_white):

    fig = all_rgi.groupby('o1region')["area_km2"].sum().plot(kind='pie', cmap=cmc.managua_r.resampled(20),
                                                             y="Region",
                                                   explode=np.zeros(17) + 0.15)
    fig.set_title("Area by Region", {"weight": "bold"})
    fig.figure.tight_layout()
    fig.figure.savefig(p / "rgi_regions.png", dpi=300, transparent=True)
plt.show()

In [ ]:
len(all_rgi["o1region"].unique())

In [ ]:
np.zeros(18)

In [ ]:
fig.fi